# This code will unzip Symbiodiniceae or Coral genomes dowloaded from NCBI. Next Genomes are translated using transdecoder and then finaly run though HMMER searching for TIR domains

In [ ]:
#Unzip the symbiodinium or coral genomes
gzip -d ../input/symbiodinium/input/genome_raw/GCA_000507305.1_ASM50730v1_genomic.fna.gz 


# Run transdecoder on genome files downloaded from NCBI
Note that the output files are generated in the location of the code script. In this case the procedure
folder.

In [ ]:
#run transdecoder
#!/bash
import os
genome_dir = "../input/genome_raw"
for genome in os.listdir(genome_dir):
    if not genome.endswith(".fasta"):
        continue
    transdecoder_file_path = os.path.join(genome_dir,genome)
    print(f"Finding the longest open reading frame for {transdecoder_file_path}") 
    !TransDecoder.LongOrfs -t {transdecoder_file_path}
    #mv folder

Finding the longest open reading frame for ../input/genome_raw/S_microadriaticum_CassKB8.genome.fasta
* Running CMD: /home/tanya/anaconda3/opt/transdecoder/util/compute_base_probs.pl ../input/genome_raw/S_microadriaticum_CassKB8.genome.fasta 0 > /home/tanya/Work_files/TIR_Domains/Symbiodinium/procedure/S_microadriaticum_CassKB8.genome.fasta.transdecoder_dir/base_freqs.dat


-first extracting base frequencies, we'll need them later.


- extracting ORFs from transcripts.
-total transcripts to examine: 67937
[67900/67937] = 99.95% done    CMD: touch /home/tanya/Work_files/TIR_Domains/Symbiodinium/procedure/S_microadriaticum_CassKB8.genome.fasta.transdecoder_dir.__checkpoints_longorfs/TD.longorfs.ok


#################################
### Done preparing long ORFs.  ###
##################################

	Use file: /home/tanya/Work_files/TIR_Domains/Symbiodinium/procedure/S_microadriaticum_CassKB8.genome.fasta.transdecoder_dir/longest_orfs.pep  for Pfam and/or BlastP searches to enable hom

# Move the files from the procedure folder to input/genomes
# Create a folder to put the "longest_orfs.pep" files into and move each over, changing adding the 
# coral or Symbiodinium name to the front.
The "longest_orfs.pep file is found in the folder starting with the species and ending with 
"transdecoder_dir".

In [ ]:
#example moving script
!cp ../procedure/genome_raw/GCA_000507305.1_ASM50730v1_genomic.fna.transdecoder_dir/longest_orfs.pep ../input/genomes/B_minutum.fasta

# Run HMMER

Build a HMMER profile using the TIR domain from Pfam

In [1]:
#!/bash
import os
pfam_domain_dir = "../input/domains"
output_dir = "../output"
for domain in os.listdir(pfam_domain_dir):
    if not domain.endswith(".sto"):
        continue
    pfam_domain_filepath = os.path.join(pfam_domain_dir, domain)
    print("Building a HMM profile from: {pfam_domain_filepath}")
    #need to split the file name so that I can get a file without the extention
    base_filename = os.path.splitext(domain)[0]
    #build the hmmer profile for each domain in the folder and save the file
    output_filepath = os.path.join(output_dir, f"{base_filename}_profile.hmm")
    !echo "Building HMM profile: {output_filepath}"
    !hmmbuild {output_filepath} {pfam_domain_filepath}

Building a HMM profile from: {pfam_domain_filepath}
Building HMM profile: ../output/PF01582_full_up_profile.hmm
# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             ../input/domains/PF01582_full_up.sto
# output HMM file:                  ../output/PF01582_full_up_profile.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     PF01582_full_up      13614  1188   299    50.60  0.590 

# CPU time: 0.65u 0.03s 00:00:00.68 Elapsed: 00:00:00.74


In [1]:
import os
#search transdecoder file for TIR domains and save them
genome_dir = "../input/genomes/longest_orfs/transdecoder_part1"
hmm_dir = "../output/"
output_dir ="../output"
for hmm_file in os.listdir(hmm_dir):
    if not hmm_file.endswith(".hmm"):
            continue
    hmm_filepath = os.path.join(hmm_dir,hmm_file)
 
    for genome_file in os.listdir(genome_dir):
        if not genome_file.endswith(".pep"):
            continue
        symbio_pep_filepath=os.path.join(genome_dir,genome_file)
        
        print(f"Running HMM search on genome {symbio_pep_filepath}")
        print(f"Running HMM search using HMM file {hmm_filepath}")
       
        symbio_pep_split = os.path.splitext(genome_file)[0]
        
        output_file = f"{symbio_pep_split}_hmmer_results.txt"
        output_filepath = os.path.join(output_dir, output_file)
        print(f"Generating output file path {output_filepath}")
        !hmmsearch --tblout {output_filepath} {hmm_filepath} {symbio_pep_filepath}


Running HMM search on genome ../input/genomes/longest_orfs/transdecoder_part1/S_tridacnidorum_longest_orfs.pep
Running HMM search using HMM file ../output/PF01582_full_up_profile.hmm
Generating output file path ../output/S_tridacnidorum_longest_orfs_hmmer_results.txt
# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ../output/PF01582_full_up_profile.hmm
# target sequence database:        ../input/genomes/longest_orfs/transdecoder_part1/S_tridacnidorum_longest_orfs.pep
# per-seq hits tabular output:     ../output/S_tridacnidorum_longest_orfs_hmmer_results.txt
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       PF01582_full_up  [M=299]
Scores for complete sequences (score includes al